In [1]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2
from imutils import paths
import os

Using TensorFlow backend.


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
# load the trained convolutional neural network
print("[INFO] loading network...")
model = load_model("/content/gdrive/My Drive/Colab Notebooks/breast_cancer_classification.model")

[INFO] loading network...


W0825 10:00:23.597865 140440993818496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 10:00:23.658559 140440993818496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0825 10:00:23.714990 140440993818496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0825 10:00:23.967149 140440993818496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0825 10:00:23.968431 140440993818496 deprecation_

In [18]:
testDatasetFolder = "/content/gdrive/My Drive/Colab Notebooks/mass_test_image_png"
imagePaths = sorted(list(paths.list_images(testDatasetFolder)))
print("Found ", len(imagePaths), " test image")

Found  21  test image


In [19]:
accuracy_labels = []
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    orig = image.copy()

    # pre-process the image for classification
    image = cv2.resize(image, (28, 28))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    # classify the input image
    (notCancer, cancer) = model.predict(image)[0]
    
    label = imagePath.split(os.path.sep)[-1]
    label = 1 if ( cancer > notCancer and "MLO" in label) or (notCancer > cancer and "CC" in label) else 0
    accuracy_labels.append(label)
    
    # build the label
    label = "Cancer" if cancer > notCancer else "Not Cancer"
    proba = cancer if cancer > notCancer else notCancer
    label = "{}: {:.2f}%".format(label, proba * 100)

    # draw the label on the image
    output = imutils.resize(orig, width=400)
    cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
        0.7, (0, 255, 0), 2)
    cv2.imwrite("/content/gdrive/My Drive/Colab Notebooks/outputs/"+imagePath.split(os.path.sep)[-1]+"-output.png", output)
    
    label = "{}, {}: {:.2f}%".format(imagePath, label, proba * 100)
    print("Path:", label)

Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00017_LEFT_CC-000000.png, Not Cancer: 99.08%: 99.08%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00156_RIGHT_MLO-000000.png, Cancer: 95.03%: 95.03%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00171_RIGHT_CC-000000.png, Not Cancer: 91.29%: 91.29%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00200_RIGHT_MLO-000000.png, Cancer: 99.53%: 99.53%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00202_RIGHT_CC-000000.png, Not Cancer: 99.98%: 99.98%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00203_LEFT_CC-000000.png, Not Cancer: 99.71%: 99.71%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_P_00324_RIGHT_CC-000000.png, Not Cancer: 93.04%: 93.04%
Path: /content/gdrive/My Drive/Colab Notebooks/mass_test_image_png/Mass-Test_

In [11]:
total_count = np.size(accuracy_labels)
print("Total Count:", total_count)
wrong_prediction_count = total_count - np.count_nonzero(accuracy_labels)
print("Wrong prediction count:", wrong_prediction_count)
print("Rate", (total_count - wrong_prediction_count)/total_count * 100)

Total Count: 7
Wrong prediction count: 0
Rate 100.0
